In [9]:
# https://bioconductor.org/packages/release/bioc/html/DESeq2.html
# https://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html

In [10]:
# if (!require("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")
# BiocManager::install("DESeq2")

In [137]:
library(DESeq2)
setwd("/disk0/sm/bulk/test")

In [138]:
library("edgeR")
# args<-commandArgs(trailingOnly=TRUE);
# Name <- args[1];
# Name <- 'H3122-LR_ctrl-1'

setwd("/disk0/sm/bulk/test/output")
cfiles <- list.files("/disk0/sm/bulk/test/output", pattern='ReadsPerGene.out.tab', recursive=TRUE)
cfiles

# non strand option. if strand specific, columns = c(1,4)
counts <- readDGE(cfiles,columns=c(1,4))
a <- unlist(strsplit(colnames(counts), split='/'))
colnames(counts) <- a[seq(1,length(a),by=2)]

noint = rownames(counts) %in% 
c("N_unmapped","N_multimapping","N_noFeature","N_ambiguous")
# CRITICAL STEP In edgeR, it is recommended to remove features without at least 1 read per million in n of the 
# samples, where n is the size of the smallest group of replicates (here, n = 3 for the knockdown group).
cpms = cpm(counts)
keep = rowSums(cpms > 1) >= 1 & !noint
counts = counts[keep,]

[1] "case1/case1_ReadsPerGene.out.tab"      
[2] "case2/case2_ReadsPerGene.out.tab"      
[3] "case3/case3_ReadsPerGene.out.tab"      
[4] "control1/control1_ReadsPerGene.out.tab"
[5] "control2/control2_ReadsPerGene.out.tab"
[6] "control3/control3_ReadsPerGene.out.tab"

In [213]:
sampletable <- counts$samples
rownames(sampletable) <- sampletable$SampleName
count_matrix <- counts$counts
sampletable$norm.factors <- c('case', 'case', 'case', 'control', 'control', 'control')
sampletable$lib.size <- c(1,1,1,0,0,0)
# a <- sampletable[c(6, 5, 4, 3, 2, 1), ]
# sampletable <- a
# head(sampletable)

In [214]:
se_star_matrix <- DESeqDataSetFromMatrix(countData = count_matrix,
    colData = sampletable,
    design = ~ norm.factors)
# se_star <- DESeqDataSetFromHTSeqCount(sampleTable = sampletable,
#                         directory = "case1",
#                         design = ~ Time)

converting counts to integer mode



Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


In [215]:
se_star_matrix <- se_star_matrix[rowSums(counts(se_star_matrix)) > 10, ]

In [216]:
se_star_matrix2 <- DESeq(se_star_matrix)

estimating size factors

estimating dispersions

gene-wise dispersion estimates



mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [217]:
norm_counts <- log2(counts(se_star_matrix2, normalized = TRUE)+1)

In [218]:
write.table(norm_counts, "/disk0/sm/bulk/test/normalized_counts.txt", quote=F, col.names=T, row.names=T, sep="\t")

In [219]:
vsd <- vst(se_star_matrix2)     # variance-stanbilizing transformation

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.



In [220]:
rld <- rlog(se_star_matrix2)    # transforms the count data to the log2 scale

In [221]:
library(pheatmap)

In [222]:
as.matrix(dist(t(assay(vsd))))

,case1,case2,case3,control1,control2,control3
case1,0.0000,288.9961,307.0214,314.2909,955.1015,306.6545
case2,288.9961,0.0000,276.8983,292.0986,940.3736,287.5145
case3,307.0214,276.8983,0.0000,315.6280,944.8326,312.4358
control1,314.2909,292.0986,315.6280,0.0000,950.0484,259.7323
control2,955.1015,940.3736,944.8326,950.0484,0.0000,941.5347
control3,306.6545,287.5145,312.4358,259.7323,941.5347,0.0000


In [223]:
sampleDistMatrix <- as.matrix(dist(t(assay(vsd))))[c('case1', 'case2', 'case3', 'control1', 'control3'), c('case1', 'case2', 'case3', 'control1', 'control3')]

In [224]:
sampleDistMatrix

,case1,case2,case3,control1,control3
case1,0.0000,288.9961,307.0214,314.2909,306.6545
case2,288.9961,0.0000,276.8983,292.0986,287.5145
case3,307.0214,276.8983,0.0000,315.6280,312.4358
control1,314.2909,292.0986,315.6280,0.0000,259.7323
control3,306.6545,287.5145,312.4358,259.7323,0.0000


In [225]:
head(vsd)

class: DESeqTransform 
dim: 6 6 
metadata(1): version
assays(1): ''
rownames(6): ENSG00000279457.4 ENSG00000225630.1 ... ENSG00000248527.1
  ENSG00000291215.1
rowData names(22): baseMean baseVar ... maxCooks dispFit
colnames(6): case1 case2 ... control2 control3
colData names(5): files group lib.size norm.factors sizeFactor

In [262]:
# png("/disk0/sm/bulk/test/sample_distance_heatmap_star.png")
  # pheatmap(sampleDistMatrix)
  # close PNG file after writing figure in it
# dev.off() 
# pheatmap(sampleDistMatrix)

In [227]:
# png("PCA_star.png")
# plotPCA(object = vsd,
# 		intgroup = "norm.factors")
# dev.off()

In [292]:
# as.matrix(assay(vsd))
de <- results(object = se_star_matrix2, name="norm.factors", contrast=c('norm.factors', 'case', 'control'))

In [298]:
resultsNames(se_star_matrix2)

[1] "Intercept"                    "norm.factors_control_vs_case"

In [309]:
install.packages('ashr')

Installing package into ‘/home/sm/R/x86_64-pc-linux-gnu-library/4.3’
(as ‘lib’ is unspecified)

also installing the dependencies ‘irlba’, ‘truncnorm’, ‘mixsqp’, ‘SQUAREM’, ‘etrunct’, ‘invgamma’




In [311]:
library(apeglm)
library(ashr)
de_shrink <- lfcShrink(dds = se_star_matrix2,
    # coef=2,
    contrast=c('norm.factors', 'case', 'control'),
    type="ashr")

using 'ashr' for LFC shrinkage. If used in published research, please cite:
    Stephens, M. (2016) False discovery rates: a new deal. Biostatistics, 18:2.
    https://doi.org/10.1093/biostatistics/kxw041



In [312]:
shrink <- na.omit(de_shrink)

In [321]:
shrin <- subset(shrink, abs(shrink$log2FoldChange)>1 & shrink$pvalue<0.05)

In [322]:
head(shrin)

log2 fold change (MMSE): norm.factors case vs control 
Wald test p-value: norm.factors case vs control 
DataFrame with 6 rows and 5 columns
                    baseMean log2FoldChange     lfcSE      pvalue        padj
                   <numeric>      <numeric> <numeric>   <numeric>   <numeric>
ENSG00000272004.1   562.3735       11.34544  1.430363 2.05163e-16 1.04927e-12
ENSG00000188257.12   13.5760       -4.06561  2.481164 1.52704e-05 1.46434e-02
ENSG00000133048.13 1797.6411       -1.47868  1.117525 8.58405e-05 5.48771e-02
ENSG00000013016.16   59.2533       -3.64563  0.839162 5.69090e-07 8.73154e-04
ENSG00000115392.13   32.4119        1.07092  1.297707 2.72425e-04 1.49279e-01
ENSG00000271474.1    14.3307        2.87730  1.573138 2.72618e-05 2.32376e-02

In [323]:
write.table(shrin, "/disk0/sm/bulk/test/deseq2_results.txt", quote=F, col.names=T, row.names=T, sep="\t")
write.table(shrink, "/disk0/sm/bulk/test/deseq2_results_total.txt", quote=F, col.names=T, row.names=T, sep="\t")